In [363]:
import networkx as nx
import matplotlib.pyplot as plt
import random
import pandas as pd
import numpy as np

In [365]:
def get_graph(filename, b_enforce_random = False):
    
    if b_enforce_random:
        random_start = 0
        random_end = 5000
        random_target = random_start + (random_end - random_start) / 2
    
    f = open(data_file, 'r')
    g = nx.DiGraph()
    
    for line in f:
        year, from_case, to_case = line[:-1].split(',')
        
        if (b_enforce_random == False) or (random_target == random.randrange(random_start, random_end)):
            g.add_edge(from_case, to_case)
            g.node[from_case]['year']=int(year)
            g.node[from_case]['ifCounted']=False
            #g.node[from_case]['traversed']=False
            g.node[from_case]['isMemer']=False
            g.node[from_case]['citesMemer']=False
            g.node[to_case]['ifCounted']=False
            #g.node[to_case]['traversed']=False
            g.node[to_case]['isMemer']=False
            g.node[to_case]['citesMemer']=False
            
    f.close()
    
    return g
    

def display_graph(graph):
    
    print graph.nodes()


def topological_sort(graph):
    """
    Sorts graph by year.
    Argument: graph
    Returns: a list of nodes of the graph sorted by dependencies
    """
    
    return nx.topological_sort(graph)


In [366]:
data_file = '../../raw/graph_stripped.csv'

g = get_graph(data_file, True)

#print 'Graph node order:'
#display_graph(g)

In [367]:
print 'Graph node order:'
display_graph(g)

Graph node order:
['X3J2AV', 'XA6RJS', 'X1C5BLQ003', 'X39M08', 'X3M8VG', 'X3TSL7', 'X129TK8003', 'X13CQTI003', 'X1CGE0E003', 'X3T0C6', 'XE7D5Q', 'XABDTC', 'XFJCUB', 'X3T0MG', 'XAC3QT', 'X3UB1D', 'X18FVIO003', 'X3N3D9', 'X120PT6003', 'X43S8C', 'X3I0GT', 'XACD47', 'X3P8VQ', 'XACD45', 'X3O9CP', 'X2NPGJ', 'XABIEC', 'X36B6R', 'XFH3A3', 'X129858003', 'X4MSH7', 'X4K9KA', 'X4K9KD', 'XABKID', 'X46G1J', 'X3ILNA', 'X5481B', 'X4AD3J', 'X4VHDA', 'X18PD2O003', 'X15EC10003', 'X3UMAS', 'X18N7BO003', 'X1ETR4M003', 'XFL9K9', 'X3SFLI', 'X4AACN', 'X3J0CM', 'X184HF4003', 'XABDFQ', 'X3F3M2', 'X3IFPV', 'X515SP', 'X47QMK', 'X4PPPK', 'XGA30R', 'X3NS3A', 'X6B6QU', 'XAFHK6', 'XN85EVQNB5G0', 'X18N7BG003', 'XACCNI', 'X40LS4', 'XABJ2P', 'XABU5B', 'X3J3U1', 'X3FE6K', 'X4PSNO', 'XABJ2D', 'X3T4MF', 'X30ITN', 'X36DJT', 'XABHAE', 'X13GF84003', 'X4SG3I', 'X3VFVI', 'X3PDMM', 'X45BOD', 'X1FI3PK003', 'X48492', 'XABQ5E', 'X1L2AMM003', 'XANN4D', 'X517UQ', 'X46892', 'X4Q47K', 'X1AJESI003', 'X435PU', 'X3U8QE', 'X3IRAA', 'X30HK4

In [368]:
sorted_list = topological_sort(g)

print 'Graph node order AFTER sorting:'
print sorted_list

Graph node order AFTER sorting:
['X3U1T6', 'X16KTBE003', 'XFITLG', 'X3A7E7', 'X1DP3FG003', 'X1CDEDE003', 'X3220I', 'X3I6HG', 'X1KGK2C003', 'X153OOG003', 'XGA2V5', 'X4O2BI', 'XAC7M0', 'X3IJPE', 'X3UMSH', 'XAFL83', 'X1GJ91C003', 'X3PP8V', 'X1H03RA003', 'X1GVQM8003', 'X41NQQ', 'XAFHE4', 'X3IRN2', 'X547IV', 'X42S8B', 'XAFN6L', 'X1BDN0U003', 'X4A5CP', 'X15F88O003', 'X1I2O2A003', 'X39FSJ', 'X3N0P6', 'X1DDJDE003', 'X3OT5V', 'X3N219', 'X42MT4', 'X51DI8', 'X3MS6R', 'X431O3', 'X6CLL7', 'X1350I0003', 'X358GC', 'XARFJVQNB5G0', 'X4SCOK', 'X4APD9', 'X3I6CD', 'X1ED8O6003', 'X12KLUK003', 'X3CT8R', 'X1AR0G0N', 'X6B7CE', 'X3H1UP', 'X519P0', 'X3FJ00', 'X3T2BD', 'X4Q4AE', 'X12A8V4003', 'XAFGD2', 'X1G256C003', 'X180Q1M003', 'X3ICM6', 'X42AM2', 'X37OV2', 'X3IHM7', 'X1GGO6ON', 'X1C1HCQ003', 'X9U7LF', 'XFLKKB', 'XFJCDC', 'X179DNA003', 'X3U88N', 'X30LT5', 'X1JGGUA003', 'X3O2EC', 'X4Q4A9', 'X3BS3L', 'X3F8VA', 'XACCON', 'X1GSSMM003', 'X3OORM', 'X1E1JG8003', 'X3A79P', 'XABO0V', 'X4A1PU', 'X1OL10ON', 'X3I67P', 'X4

In [369]:
'XDOJINQNB5G0' in sorted_list

False

In [370]:
nx.edges(g,'X4995T')

[]

In [371]:
# Accessing children of nodes in sorted list

for node in sorted_list:
    
    print nx.edges(g, node)

[('X3U1T6', 'X3TNV1')]
[('X16KTBE003', 'X6CP0C')]
[('XFITLG', 'XFL6N0')]
[('X3A7E7', 'X3SRMN')]
[('X1DP3FG003', 'X2O4JC')]
[('X1CDEDE003', 'X19HUUQ003')]
[('X3220I', 'X2NP5O')]
[('X3I6HG', 'X3H1UP')]
[('X1KGK2C003', 'XAFVT8')]
[('X153OOG003', 'X6CUO0')]
[('XGA2V5', 'X3IV5R')]
[('X4O2BI', 'XAC6UH')]
[('XAC7M0', 'X4AD3J')]
[('X3IJPE', 'X3I6CD')]
[('X3UMSH', 'XABJHI')]
[('XAFL83', 'X3I1GC')]
[('X1GJ91C003', 'X1BMRFS003')]
[('X3PP8V', 'X4A9BR')]
[('X1H03RA003', 'X3FK14')]
[('X1GVQM8003', 'X4QCQO')]
[('X41NQQ', 'X41IOU')]
[('XAFHE4', 'X3TQNC')]
[('X3IRN2', 'X3PO5I')]
[('X547IV', 'X31U7O')]
[('X42S8B', 'X3T0SE')]
[('XAFN6L', 'X48492')]
[('X1BDN0U003', 'X3LREB')]
[('X4A5CP', 'XABJAF')]
[('X15F88O003', 'X3FJGK')]
[('X1I2O2A003', 'X12GLQ4003')]
[('X39FSJ', 'X4SG3I')]
[('X3N0P6', 'X4PU9Q')]
[('X1DDJDE003', 'X1CJOF8003')]
[('X3OT5V', 'X42MT4')]
[('X3N219', 'X3GUUT')]
[]
[('X51DI8', 'X3IO44')]
[('X3MS6R', 'X46AHO')]
[('X431O3', 'X464PK')]
[('X6CLL7', 'X3CT8R')]
[('X1350I0003', 'X6B8PS')]
[('X358GC

In [372]:
data_dir='/scratch/sv1239/projects/mlcs/mlcs16/n_grams/'

In [373]:
import os
#for filename in os.listdir(data_dir):
test=pd.read_csv(data_dir +'XFJD1I.txt', sep=",",header=None)
test.columns = ['text','count']

In [374]:
test.head()

,text,count
0,not intended,1
1,perfecting the title,1
2,done by,1
3,act was,1
4,was built as,1


In [375]:
'not intended' in test['text'].tolist()

True

In [24]:
gram_graph=nx.DiGraph()
f = open(data_dir +'XFJD1I.txt', 'r')

In [29]:
'XFKLAA.txt'[:-4]

'XFKLAA'

In [48]:
gram_dict={}

In [33]:
gram_dict={'a':{'ab':1,'aab':2},'b':{'bc':3}}

In [49]:
gram_dict2={'a':['ab','aab'],'b':['bc']}

In [57]:
gram_dict2

{'a': ['ab', 'aab', 'aac', 'aacd'], 'b': ['bc']}

In [56]:
gram_dict2['a'].append('aacd')

In [42]:
gram_dict['c']={}

In [47]:
gram_dict

{'a': {'aab': 2, 'ab': 1}, 'b': {'bc': 3}, 'c': {'ca': 3, 'cb': 2}}

In [46]:
gram_dict['c']['cb']=2

In [40]:
gram_dict.has_key('bc')

False

In [377]:
import os
gram_dict={}
test=['XFKEIQ.txt','XFKK8M.txt','XFLD3P.txt']
for filename in os.listdir(data_dir):
    f = open(data_dir+filename, 'r')
    name = filename[:-4]
    gram_dict[name]={}
    for line in f:
        #print line
        #print line
        text, count = line.rsplit(',',1)
        count=int(count[:-2])
        if count>1:
            gram_dict[name][text]=count
    f.close()

In [378]:
gram_dict.keys()

['XFLCRA',
 'XFKMOB',
 'XFKEI5',
 'XFVA4E',
 'XFLD4U',
 'XFJD8T',
 'XFKB8A',
 'XFJD1I',
 'XDHGKBQNB5G0',
 'XFJD1N',
 'XFKLAA',
 'XFKEJH',
 'XFKEJF',
 'XFJD09',
 'XFKEJA',
 'XFJV7I',
 'XFKL0G',
 'XFKEJT',
 'XFKKVF',
 'XFJCU3',
 'XFKMQU',
 'XFKL8R',
 'XFJD0T',
 'XFL6HB',
 'XJ8KC8N',
 'XFJD0F',
 'XFLCDV',
 'XFLDA9',
 'XOKFRRQNB5G0',
 'XFKKR7',
 'XFSD8E',
 'XFKEJ0',
 'XDOL47QNB5G0',
 'XFKEIQ',
 'XFJV6T',
 'XB0PLBQNB5G0',
 'XFKEMC',
 'XFLCK7',
 'XFKEMD',
 'XDOL0JQNB5G0',
 'XFL7IV',
 'XFKL34',
 'XFLJFE',
 'XFKB64',
 'XFKKTL',
 'XFKEK8',
 'XFJD3M',
 'XFLCIP',
 'XFKKLC',
 'XFKEK6',
 'XFKEK1',
 'XFJVAM',
 'XFKEHK',
 'XFKEHL',
 'XFLD3P',
 'XFKC1R',
 'XFJD24',
 'XFKLCE',
 'XOKFQBQNB5G0',
 'XFKEHE',
 'XFJD23',
 'XFKEL4',
 'XDHGKFQNB5G0',
 'XFKEIC',
 'XFJD80',
 'XFLDI9',
 'XFKISA',
 'XFLD5A',
 'XFLCQC',
 'XFLCQD',
 'XDOJINQNB5G0',
 'XFLCQJ',
 'XFKK8M',
 'XFJD2P',
 'XFLCH5',
 'X19A77QNB5G0',
 'XFKL0R',
 'X18KAVQNB5G0',
 'XFLCV0',
 'XFJUR9',
 'XFKEH3',
 'XFKEIM',
 'XFL7R1',
 'XFKEGI',
 'XFLDFC',
 'XF

In [379]:
haskeys=[]

In [380]:
for nd in gram_dict.keys():
    if nd in g.nodes():
        print nd

In [381]:
for nd in g.nodes():
    if gram_dict.has_key(nd):
        print nd
        haskeys.append(nd)

In [382]:
haskeys

[]

In [383]:
for node in g.nodes():
    if len(g.successors(node))>1:
        print node
        print g.successors(node)

XFL6JH
['XBB7IBQNB5G0', 'XL2H37']
X510P5
['X3GH5R', 'XCMD4VQNB5G0']
X1H4OA2003
['X3AB97', 'X3MHTA']
X1H75VC003
['XFP2JF', 'XAI44M']
X3NQ9G
['X42KTJ', 'X3H464']


In [438]:
def gramScorer(gram,sorted_list,graph,gram_dict):
    """
    Calculates meme score for each ngram.
    Arguments:
    gram: an n-gram or phrase for which meme score to be calculated
    sorted_list: a sorted list of all nodes in graph
    graph: a NetworkX DAG of nodes/cases with edges
    gram_dict: dict storing n-grams
    Returns:
    meme_score: final meme_score of the n-gram
"""
    graph_size = len(sorted_list) #number of nodes
    dm2m=0
    d2m=0
    dm2n=0
    d2n=0
    memers=0
    for node in sorted_list: #list of nodes sorted topologically
        #every node stores 3 vars - isMemer, citesMemer, ifCounted (in sum)
        if not graph.node[node]['ifCounted']: #if not counted in sum
            if node in gram_dict.keys():
                if gram_dict[node].has_key(gram): #if gram present
                    memers+=1 #has meme
                    graph.node[node]['isMemer']=True
            graph.node[node]['ifCounted']=True
            if len(graph.successors(node))>0: #has children (citers)
#                 if len(graph.successors(node))==1: #just one child (citer)
#                     child=graph.successors(node)[0]
#                     graph.node[node]['citesMemer']=graph.node[child]['isMemer']
#                 else:
                for child in graph.successors_iter(node):
                    if graph.node[child]['isMemer']==True:
                        graph.node[node]['citesMemer']=True
                        break
#             if not graph.node['node']['traversed']: #not been traversed
#                 graph.node['node']['traversed']=True
#                 if gram_dict[node].has_key(gram): #if gram present
#                     memers+=1 #has meme
#                     graph.node['node']['isMemer']=True
            if graph.node[node]['isMemer']==True and graph.node[node]['citesMemer']==True:
                dm2m+=1 #memer cites memer
                d2m+=1 #cites memer
                graph.node[node]['ifCounted']=True
            elif graph.node[node]['isMemer']==True and graph.node[node]['citesMemer']==False:
                dm2n+=1 #memer cites non-memers
                d2n+=1 #cites non-memers
                graph.node[node]['ifCounted']=True
            elif graph.node[node]['isMemer']==False and graph.node[node]['citesMemer']==True:
                d2m+=1 #cites memers
            elif graph.node[node]['isMemer']==False and graph.node[node]['citesMemer']==False:
                d2n+=1 #cites non-memers
    print dm2m
    print d2m
    print dm2n
    print d2n
    
    prop_score = ((dm2m)/(d2m+3.))/((dm2n+3.)/(d2n+3.))
    freq = memers/graph_size
    meme_score = prop_score*freq
    print prop_score
    print freq
    print meme_score
    return meme_score

In [385]:
for node in sorted_list:
    if gram_dict.has_key(node):
        print node
print "there's nothing"

there's nothing


In [386]:
for i in ['XDOJINQNB5G0']:
    #assert i.year <= sorted.next.year #check that time-sorted
    for gram in gram_dict[i]:
        gramScore = gramScorer(gram,sorted_list,g)
        score_dict[i][gram] = gramScore

TypeError: gramScorer() takes exactly 4 arguments (3 given)

### Since we don't have n-grams for nodes in graph_stripped, we'll use dummy data.

In [421]:
dummy_dict={}

In [422]:
len(gram_dict.keys())

176

In [439]:
for i in range(len(gram_dict.keys())):
    state=random.randrange(0,len(sorted_list))
    while sorted_list[state] in dummy_dict.keys():
        state=random.randrange(0,len(sorted_list))
    dummy_dict[sorted_list[state]]=gram_dict.values()[i]

In [440]:
len(dummy_dict)

352

In [441]:
cnt=0
for go in dummy_dict.keys():
    if go in sorted_list:
        cnt=cnt+1
print cnt

352


In [442]:
print dummy_dict.has_key('X3U1T6')
print 'X3U1T6' in sorted_list
print 'X3U1T6' in g.nodes()

False
True
True


In [443]:
dummy_dict.keys()[0]

'X1IIA7O003'

In [444]:
testset=[dummy_dict.keys()[0]]
score_dict={}
for i in testset:
    score_dict[i] = {}
    for gram in dummy_dict[i]:
        gramScore = gramScorer(gram,sorted_list,g,dummy_dict)
        score_dict[i][gram] = gramScore
        print gram,gramScore

0
0
0
0
0.0
0
0.0
saved from 0.0
0
0
0
0
0.0
0
0.0
it did not 0.0
0
0
0
0
0.0
0
0.0
lightly aground 0.0
0
0
0
0
0.0
0
0.0
found that 0.0
0
0
0
0
0.0
0
0.0
service had been 0.0
0
0
0
0
0.0
0
0.0
have been 0.0
0
0
0
0
0.0
0
0.0
service had 0.0
0
0
0
0
0.0
0
0.0
did not 0.0
0
0
0
0
0.0
0
0.0
salvage awarded 0.0
0
0
0
0
0.0
0
0.0
share with 0.0
0
0
0
0
0.0
0
0.0
been rendered 0.0
0
0
0
0
0.0
0
0.0
they did 0.0
0
0
0
0
0.0
0
0.0
it did 0.0
0
0
0
0
0.0
0
0.0
vessel was 0.0
0
0
0
0
0.0
0
0.0
having been 0.0
0
0
0
0
0.0
0
0.0
had been 0.0
0
0
0
0
0.0
0
0.0
where it 0.0
0
0
0
0
0.0
0
0.0
beneficial service had 0.0
0
0
0
0
0.0
0
0.0
had been rendered 0.0
0
0
0
0
0.0
0
0.0
had gone 0.0
0
0
0
0
0.0
0
0.0
has been 0.0
0
0
0
0
0.0
0
0.0
beneficial service 0.0
0
0
0
0
0.0
0
0.0
I consider 0.0
0
0
0
0
0.0
0
0.0
they did not 0.0
0
0
0
0
0.0
0
0.0
class of cases 0.0
0
0
0
0
0.0
0
0.0
was not 0.0


In [446]:
score_dict

{'X1IIA7O003': {'I consider': 0.0,
  'been rendered': 0.0,
  'beneficial service': 0.0,
  'beneficial service had': 0.0,
  'class of cases': 0.0,
  'did not': 0.0,
  'found that': 0.0,
  'had been': 0.0,
  'had been rendered': 0.0,
  'had gone': 0.0,
  'has been': 0.0,
  'have been': 0.0,
  'having been': 0.0,
  'it did': 0.0,
  'it did not': 0.0,
  'lightly aground': 0.0,
  'salvage awarded': 0.0,
  'saved from': 0.0,
  'service had': 0.0,
  'service had been': 0.0,
  'share with': 0.0,
  'they did': 0.0,
  'they did not': 0.0,
  'vessel was': 0.0,
  'was not': 0.0,
  'where it': 0.0}}